In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("bySQL").getOrCreate()

In [0]:
#types of trnasformation:
# 1. narrow transformations = sort by, union, filter/where, distinct [not required shuffling data]
# 2. wide transformations = group by, joins (left, right, full outer joins, inner joins, cross joins), agg, pivot, sort, repartition. [required shuffling data]

In [0]:
parquet_df = spark.read.parquet("/FileStore/tables/Titanic.parquet")
parquet_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [0]:
#operations
parquet_df.createOrReplaceTempView("titanic_table")

In [0]:
spark.sql("""
          select PassengerID, Sex, Ticket, Fare from titanic_table 
          where Sex = "male" and  Fare > 100
          limit 5
          """).show()

+-----------+----+--------+--------+
|PassengerID| Sex|  Ticket|    Fare|
+-----------+----+--------+--------+
|         28|male|   19950|   263.0|
|        119|male|PC 17558|247.5208|
|        306|male|  113781|  151.55|
|        333|male|PC 17582|153.4625|
|        374|male|PC 17760|135.6333|
+-----------+----+--------+--------+



In [0]:
#sql tranformations:
#clause: select, where, group by, order by, having
#functions: string function, arthematic functions, date functions
#aggregate functions: min, max, sum, avg, count
#joins: inner join, left, right, full outer joins, cross joins
#

In [0]:
# using sql
parquet_df.createOrReplaceTempView("titanic_table")
spark.sql("select Sex, Fare, Ticket from titanic_table").show(5)
#using dataframe
parquet_df.select("Sex", "Fare", "Ticket").show(5)

+------+-------+----------------+
|   Sex|   Fare|          Ticket|
+------+-------+----------------+
|  male|   7.25|       A/5 21171|
|female|71.2833|        PC 17599|
|female|  7.925|STON/O2. 3101282|
|female|   53.1|          113803|
|  male|   8.05|          373450|
+------+-------+----------------+
only showing top 5 rows

+------+-------+----------------+
|   Sex|   Fare|          Ticket|
+------+-------+----------------+
|  male|   7.25|       A/5 21171|
|female|71.2833|        PC 17599|
|female|  7.925|STON/O2. 3101282|
|female|   53.1|          113803|
|  male|   8.05|          373450|
+------+-------+----------------+
only showing top 5 rows



In [0]:
# using sql
parquet_df.createOrReplaceTempView("titanic_table")
spark.sql("select * from titanic_table where Fare > 200").show(5)
#using dataframe
parquet_df.where("Fare > 200") .show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+
|         28|       0|     1|Fortune, Mr. Char...|  male|19.0|    3|    2|   19950|   263.0|C23 C25 C27|       S|
|         89|       1|     1|Fortune, Miss. Ma...|female|23.0|    3|    2|   19950|   263.0|C23 C25 C27|       S|
|        119|       0|     1|Baxter, Mr. Quigg...|  male|24.0|    0|    1|PC 17558|247.5208|    B58 B60|       C|
|        259|       1|     1|    Ward, Miss. Anna|female|35.0|    0|    0|PC 17755|512.3292|       null|       C|
|        300|       1|     1|Baxter, Mrs. Jame...|female|50.0|    0|    1|PC 17558|247.5208|    B58 B60|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-----

In [0]:
# using sql
parquet_df.createOrReplaceTempView("titanic_table")
spark.sql("select Sex, count(*) from titanic_table group by Sex").show(5)
#using dataframe
parquet_df.groupBy("Sex").count().show(5)

+------+--------+
|   Sex|count(1)|
+------+--------+
|female|     314|
|  male|     577|
+------+--------+

+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+



In [0]:
# using sql
parquet_df.createOrReplaceTempView("titanic_table")
spark.sql("select Name from titanic_table order by Name").show(5)
#using dataframe
parquet_df.select("Name").orderBy("Name").show(5)

+--------------------+
|                Name|
+--------------------+
| Abbing, Mr. Anthony|
|Abbott, Mr. Rossm...|
|Abbott, Mrs. Stan...|
| Abelson, Mr. Samuel|
|Abelson, Mrs. Sam...|
+--------------------+
only showing top 5 rows

+--------------------+
|                Name|
+--------------------+
| Abbing, Mr. Anthony|
|Abbott, Mr. Rossm...|
|Abbott, Mrs. Stan...|
| Abelson, Mr. Samuel|
|Abelson, Mrs. Sam...|
+--------------------+
only showing top 5 rows



In [0]:
# using sql
parquet_df.createOrReplaceTempView("titanic_table")
spark.sql("select Name from titanic_table order by Name desc").show(5)
#using dataframe
parquet_df.select("Name").orderBy(col("Name").desc()).show(5)

+--------------------+
|                Name|
+--------------------+
|van Melkebeke, Mr...|
|van Billiard, Mr....|
|del Carlo, Mr. Se...|
|de Pelsmaeker, Mr...|
|de Mulder, Mr. Th...|
+--------------------+
only showing top 5 rows

+--------------------+
|                Name|
+--------------------+
|van Melkebeke, Mr...|
|van Billiard, Mr....|
|del Carlo, Mr. Se...|
|de Pelsmaeker, Mr...|
|de Mulder, Mr. Th...|
+--------------------+
only showing top 5 rows



In [0]:
# using sql
parquet_df.createOrReplaceTempView("titanic_table")
spark.sql("select Sex, count(*) from titanic_table group by Sex having Sex = 'male'").show(5)
#using dataframe
parquet_df.groupBy("Sex").count().where("Sex = 'male'").show(5)

+----+--------+
| Sex|count(1)|
+----+--------+
|male|     577|
+----+--------+

+----+-----+
| Sex|count|
+----+-----+
|male|  577|
+----+-----+

